# DNN Lab

## Objectives
-  Understand basic DNN model building process using Keras
-  Analyze model performance and capacity vs generalization tradeoff
-  Modify models to reduce overfitting and improve performance

## Exercises
-  Build a DNN model for slump Test Problem
-  Start with a model consisting of one hidden layer with 7 neurons
-  Analyze results and explore improvements to model in terms of capacity, regularization





### Step 1: Import Libraries

In [2]:
%tensorflow_version 2.x
from numpy.random import seed
seed(2)
import tensorflow as tf
from tensorflow import keras
from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.python.data import Dataset
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
print(tf.__version__)

2.1.0


### Step 2: Import Data

In [0]:

pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format

concrete_dataframe = pd.read_csv("slump_test.data", sep=",")

concrete_dataframe = concrete_dataframe.reindex(
    np.random.permutation(concrete_dataframe.index))

In [5]:
concrete_dataframe.shape[0]

103

### Step 3: Preprocess

In [0]:
def preprocess_features(concrete_dataframe):
  """Prepares input features from concrete slump test data set.

  Args:
    concrete_dataframe: A Pandas DataFrame expected to contain data
      from the concrete slump test dataset.
  Returns:
    A DataFrame that contains the features to be used for the model. 
  """
  selected_features = concrete_dataframe[
    ["Cement",
     "Slag",
     "Fly ash",
     "Water",
     "SP",
     "Coarse Aggr.",
     "Fine Aggr."]]
    
  processed_features = selected_features.copy()
  
  return processed_features

def preprocess_targets(concrete_dataframe):
  """Prepares target features (i.e., labels) from  housing data set.

  Args:
    dataframe: A Pandas DataFrame expected to contain data
      from the data set.
  Returns:
    A DataFrame that contains the target feature.
  """
  output_targets = concrete_dataframe["Compressive Strength (28-day)(Mpa)"]
  return output_targets

**Train/Validation Split**

### Step 4: Build Model

https://www.tensorflow.org/api_docs/python/tf/keras/Model

https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense

https://keras.io/optimizers/

In [8]:
training_examples.shape

#training_targets.shape

(83, 7)

**Build Model**

**Fit Model**

**Lowest Validation Error**

In [0]:
print(min(b_history.history['val_mae']))

### Step 5: Plot Results

### Predictions

In [0]:
valpreds = baseline_model.predict_on_batch(validation_examples)
print(valpreds)

In [0]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(validation_targets)

In [0]:
# Plot Weights
nfw = baseline_model.get_weights()[0][0]
y_pos = np.arange(len(nfw))
 
plt.bar(y_pos, nfw, align='center', alpha=0.5)